# Cats and Qubits

Quantum Mechanics has fundamentally changed the way we study natural phenomena on microscopic scales. However, we've only just started to realize how quantum mechanics can also revolutionize information processing and computer science. The new and promising field of quantum computing makes use of the powerful, albeit sometimes unintuitive, quantum phenomena.

Classical information processing begins with a bit, and analogously, quantum computing begins with a qubit. Information is measured in bits, where each bit will only ever be in one of two possible states (commonly referred to as "0" and "1"). A qubit is a two state quantum-system, so it's quite similar in that they only ever hold one bit of information, but the superposition principle in quantum mechanics allows a qubit to occupy any one of the continuum of states in Hilbert space - sounds like a contradiction, huh?

Remember that even though a quantum system can occupy any state in Hilbert space, the only way we can get information from a qubit is by measuring it, at which point the state will collapse to one of the two possible basis states. For simplicity, let's choose a measurement operator which commutes with the Hamiltonian of our system, so we don't have to worry about the time dependence of our system. Now we can define the two eigenstates of our measurement operator to be $|0\rangle$ and $|1\rangle$ which forms our "computational basis." (Look up what measurement in quantum mechanics means.)

So in general, the state of our qubit $|\psi \rangle$ can be written as:

$$ |\psi \rangle = \alpha |0\rangle + \beta |1\rangle $$

for any $\alpha$ and $\beta$ such that $|\alpha|^2 + |\beta|^2 = 1$.

## Quantum Circuits

Just as classical bits are processed in circuits with logic gates, qubits are processed in quantum circuits using quantum logic gates.

A very convenient framework for building, and simulating quantum circuits using python is `qiskit`, which we'll use now.

In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit_aer import AerSimulator # for simulating circuits
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In Qiskit, we can represent all of our operations in a `QuantumCircuit`. This object initializes all qubits in the $|0\rangle$ state and applies operations from left to right.

In [ ]:
qc = QuantumCircuit(1)

Your circuits can be visualized using the `draw` function.

Note - "mpl" formats the circuit visualization to make it look pretty.

In [ ]:
qc.draw("mpl")
plt.draw()

Our circuit isn't very exciting right now - let's add a Hadamard gate! Remember, a Hadamard gate puts our qubit in superposition.

For a little nuance, it transforms the $|0\rangle$ state into the $|+\rangle$ state and transforms the $|1\rangle$ state into the $|-\rangle$ state, where: $$|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$$ $$|-\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle).$$ The phase difference between the two states won't matter for measurement (we'll still have an equal chance of measuring the zero and one state), but it will be useful in future quantum algorithms. 

For the $|+\rangle$ and $|-\rangle$ states, see Section 7.2. For the Hadamard gate, see Section 8.3.

In [ ]:
qc.h(0)
qc.measure_all()
qc.draw("mpl")
plt.draw()

For a quick explanation of the circuit above:
- The red box with an H represents our Hadamard gate.
- The vertical dashed-line represents a barrier. This isn't a gate - it just helps makes the circuit more readable.
- The grey square with the dial is the measurement gate. It ends the circuit and measures that specific qubit.

Now we can simulate our circuit to do some basic quantum computing.

First, you have to choose what kind of backend to use. We'll using two different backends: the `StatevectorSampler`, which turns the circuit into a pure statevector before computation, and `Fake Sherbrooke`, that simulates a quantum computer running your circuit. The `StatevectorSampler` will run your quantum circuit without any noise, while Fake `Fake Sherbrooke`, like real quantum computers, will include noise in its computation. There are also some backends that will run your circuit on a real quantum computer (for more information check out IBMQ Experience).

With IBM's new Qiskit 1.0 update, there are a couple of steps we need to take to run our newly created quantum circuit.

First, let's create a backend. This is the quantum computer (or simulated quantum computer) that we'll be using to run our quantum circuit. To keep things clear, we're going to call the backend a sampler this time around, to differentiate the `StatevectorSampler` and the `Fake Sherbrooke`. 

Next, we can "run" our quantum circuit. Since quantum computations are inherently probabilistic, we don't just run our quantum circuit once - we run it many times to get a good idea on the probabilities of each state. The number of times (or the number of shots) we run a quantum circuit with is usually 1024, but we can change it manually. We can check this by seeing how many bitstrings we have (a bitstring is the result of running our circuit from an initial state). This is the number of times we ran the quantum circuit or, in other words, the number of outputs we got.

Finally, we'll see the number of times we got certain states using get_counts(). This will return a dictionary, with each state as the key and the number of times that state was measured as the value.

In [ ]:
sampler = StatevectorSampler()
result = sampler.run([qc]).result()
# Access result data for PUB 0
data = result[0].data

In [ ]:
bitstrings = data.meas.get_bitstrings()
print(f"The number of bitstrings is: {len(bitstrings)}")

In [ ]:
# Get counts for the classical register "meas"
counts = data.meas.get_counts()
print(f"The counts are: {counts}")
plot_histogram(counts)
plt.draw()

In [ ]:
probs = {}
total_counts = len(bitstrings)
total_prob = 0

for state in counts.keys():
    probs[state] = counts[state]/total_counts
    total_prob += counts[state]/total_counts

print(f"The probabilities of each state: {probs}")

In theory, when we make a measurement on the $$|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$$ we expect to see the zero and one state with precisly 50% probability each. However, as you can see above, this is not the case with real measurements (unless you are really lucky!).

## Cat States

Named after Schrödinger's (possibly) misfortunate cat, a cat state is an equal superposition with only two possible, very opposite outcomes.

$$ |\mathrm{🐱} \rangle = \frac{1}{\sqrt{2}} (|000\dots00 \rangle + |111\dots11 \rangle) $$

Are the individual qubits in a cat state entangled? How do you know?

[your answer here]

Starting from the default state of all zeros, let's build to circuit to create a cat state for a two qubit system. We start off by initializing our circuit.

In [ ]:
qc = QuantumCircuit(2)
qc.draw("mpl")
plt.draw()

We'll need another Hadamard - in fact, essentially all quantum circuits begin with at least one Hadamard, any ideas why?

[your answer here]

In [ ]:
qc.h(0)
qc.draw("mpl")
plt.draw()

Next, we'll use a controlled NOT gate (aka CNOT or CX), which is a very important binary gate. The CNOT gate takes a control qubit and a target qubit as input, and inverts (applies an X gate) the target qubit if and only if the control qubit is 1.

Classically, such a gate is nothing special. In the quantum realm, however, thanks to superposition, the CNOT gate allows us entangle qubits.

In [ ]:
qc.cx(0,1)
qc.measure_all()
qc.draw("mpl")
plt.draw()

The big circle with the plus sign takes the target input, while the blue dot on top takes the control input.

Now, let's simulate our circuit to compute the resulting state. This time, we're going to be using a simulation of an actual quantum computer. This backend, FakeSherbrooke, will simulate the qubits directly (including noise), so there are a couple more steps we need to take before we can run our circuit.

We need to transpile our general quantum circuit above into a circuit that fits on our target hardware. In theory, quantum computers can implement any gate you want and can entangle any pair of qubits. 

In practice, this isn't true. Quantum computers, due to size constraints, only physically connect certain qubits. If two qubits aren't directly connected, we can't entangle them directly. Real quantum hardware also uses a very specific gate set - a basis gate set. This basis gate set consists of a few, specifically tuned gates for that quantum computer. Before we can run our quantum circuit, which can have any type of gate, we need to transform those gates into the quantum hardware's basis gate set so that our backend can actually run our circuit. 

While you can look at the set of quantum gates that FakeSherbrooke uses, we will not go into detail about that here. The main thing to remember is that FakeSherbrooke models our circuit above (with the Hadamard and CNOT gates) including noise.

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit import transpile

backend = FakeSherbrooke()
# Transpile circuit
transpiled_circuit = transpile(qc, backend)
# Run using sampler
job = backend.run(transpiled_circuit)
counts = job.result().get_counts()
print(f"The counts are: {counts}")
plot_histogram(counts)
plt.draw()

Remember the dimensions are ordered: $00$, $01$, $10$, $11$, so the state we are left with after applying our circuit can only have two outcomes: $00$ or $11$ - or in cat speak - very dead or very alive.

Notice that there are a few counts where the measurement gives us states 01 and 10. This is due to noise - current quantum computers aren't perfect. The gates they apply might not always perfectly rotate the qubit to an equal superposition or surrounding heat might randomly flip bits. Even though we're not running our circuit on a real computer, IBM's fake backends imitate this noise, leading us to measure states that shouldn't be possible to measure.

This is why measuring over many shots is important. With only one measurement, we don't get a clear enough picture of the state of the quantum circuit to make any conclusions or to get any meaningful results.

### Problem 1: Three Qubit Cat

Build a quantum circuit that produces a 3 qubit cat state and confirm that it is a cat state by simulating your circuit using the `statevector_simulator` and printing the resulting statevector.

In other words, starting from the (default) state $| \psi \rangle = |000\rangle$, build a circuit $\hat{\mathbf{U}}$ which has the following effect:

$$ \hat{\mathbf{U}} |\psi\rangle = \frac{1}{\sqrt{2}}(|000\rangle + |111\rangle) $$

Hint: Start with the circuit above to produce a two qubit cat state.

[Your Answer Here]

### Problem 2: Super Superpositions

The real power of quantum computing comes from the parallelism. Thanks to quantum superposition states, applying a single quantum gate to $n$ qubits can affect all $2^n$ possible outcomes those qubits can have in parallel. As a result, most quantum algorithms try to take full advantage of this parallelism by starting with a state that's an equal superposition of all possible outcomes.

Build a 3 qubit quantum circuit that transforms the initial state $|000\rangle$ to an equal superposition of all 8 possible outcomes. Test your circuit using `Fake Sherbrooke` and plot the outcomes of 1024 shots in a histogram.

Hint: Also, the simplest solution does not require any gates that weren't used above.

[Your Answer Here]

### Problem 3: Bell States Galore

For now, we've only created one out of the 4 bell possible bell states. Create the other three Bell States.

$$ |B2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle) $$
$$ |B3\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle) $$
$$ |B4\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle) $$

Draw your circuits. Test your circuits using `Fake Sherbrooke` and plot the outcomes of 1024 shots in a histogram.

In [ ]:
# Your code below for |B2>





In [ ]:
# Your code below for |B3>





In [ ]:
# Your code below for |B4>


